In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
from scipy import stats
from scipy.stats import zscore
from math import radians, cos, sin, asin, sqrt
import pydot_ng as pydot
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model

import tensorflow as tf

In [55]:
depev_final = pd.read_csv('depev_final.csv')
depev_final.drop(depev_final.columns[depev_final.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
depev_final.head()

,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,FAMINCTC_A,IMPINCFLG_A,PPSU,PSTRAT,HISPALLP_A,RACEALLP_A,...,CHDEV_A,CHLEV_A,HYPEV_A,PHSTAT_A,AVAIL_A,INTV_QRT,WTFA_A,WTIA_A,HHX,POVRATTC_A
0,4,9,3,0,60000,2,2,122,3,2,...,2,1,2,3,1,1,13177.008,7601.336,H048109,2.96
1,4,9,3,0,50000,0,2,122,2,1,...,2,2,2,1,1,1,6140.552,3344.434,H027044,2.97
2,4,12,3,0,65000,1,2,122,2,1,...,2,1,1,3,1,1,9191.061,6949.498,H058855,4.28
3,4,14,5,0,120000,0,2,122,1,8,...,2,2,1,2,1,1,7900.035,6446.327,H031993,7.13
4,1,4,1,0,30000,0,2,115,2,1,...,1,1,1,4,1,1,10875.772,8646.586,H007122,1.13


In [56]:
depev_final['DEPEV_A']

0        2
1        2
2        2
3        2
4        1
        ..
31954    2
31955    2
31956    2
31957    1
31958    2
Name: DEPEV_A, Length: 31959, dtype: int64

In [57]:
depev_data = depev_final.sample(frac=1)

train = depev_data.sample(frac=0.8)
print('Train:' + str(train.shape))
validate = depev_data.sample(frac=0.1)
print('Validate:' + str(validate.shape))
test = depev_data.sample(frac=0.1)
print('Test:' + str(test.shape))

label_column = 'DEPEV_A'
depev_label = depev_data[['DEPEV_A']].to_numpy()

Train:(25567, 165)
Validate:(3196, 165)
Test:(3196, 165)


In [58]:
# Split into train val test
x_train = train.drop(['DEPEV_A'], axis=1)
y_train = train[['DEPEV_A']].to_numpy()

x_val = validate.drop(['DEPEV_A'], axis=1)
y_val = validate[['DEPEV_A']].to_numpy()

x_test = test.drop(['DEPEV_A'], axis=1)
y_test = test[['DEPEV_A']].to_numpy()

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(25567, 164)
(25567, 1)
(3196, 164)
(3196, 1)
(3196, 164)
(3196, 1)


In [59]:
xsize=164
ysize=1

t_model = Sequential()
t_model.add(Dense(100, activation="relu", input_shape=(xsize,)))
t_model.add(Dense(50, activation="relu"))
t_model.add(Dense(ysize))
t_model.compile(
    loss="mean_squared_error",
    optimizer=Adam(lr=0.001),
    metrics=[metrics.mae])

/Users/josephsun/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [60]:
epochs = 500
batch = 128

# cols = list(train.columns)
# cols.remove(label_column)
history = t_model.fit(
    x_train, y_train,
    batch_size=batch,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(x_val,y_val)
)
score = model.evaluate(x_test, y_test, verbose=0)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).